In [1]:
import os
import sys
import openmm
import openmm.app
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
#os.chdir("/srv/disk1/MYPROG/ARBALEST/OPENMM_ARROW/plugins/arrow/tests")
os.getcwd()

'/srv/disk1/MYPROG/ARBALEST/OPENMM_ARROW/plugins/arrow/tests'

In [2]:
ene_aa_ff = glob.glob("Output_AA_FF_DBL/*.ene")[0]
#ene_aa_nn_12 = glob.glob("Output_AA_NN_12_FLT/*.ene")[0]
ene_aa_nn_12 = glob.glob("Output/*.ene")[0]
df_aa_ff = pd.read_csv(ene_aa_ff,sep=r'\s+')
df_aa_nn_12 = pd.read_csv(ene_aa_nn_12,sep=r'\s+')

In [ ]:
df_aa_nn_12

,Time,Temp,EnrgPot,EnrgKin,EnrgTotal,EnrgCoul,EnrgVdW,EnrgBond,EnrgAngle,EnrgStretch,EnrgTorsion,EnrgOOP,EnrgNN,EnrgNNBnd,Virial,VirialValent,VirialNonValent,VirialNN
0,0.000,3.385412e+02,-39.106487,2.220077e+01,-1.690572e+01,-11.254647,-17.114207,2.176930,3.767734,-0.155865,-8.141166,-1.395717,-16.929417,0.0,-26.221641,-139.603242,51.792837,61.588764
1,0.001,2.870028e+02,-33.167020,1.882100e+01,-1.434602e+01,-11.229967,-17.055773,3.844604,4.117530,0.170561,-8.267083,-1.319789,-13.416472,0.0,-308.009455,-192.174151,52.351422,-168.186725
2,0.002,2.247488e+02,-28.550403,1.473852e+01,-1.381189e+01,-11.208159,-17.005137,7.091571,5.259824,0.409425,-8.389246,-1.220370,-13.546733,0.0,-223.697725,-226.627452,52.695196,-49.765468
3,0.003,1.749425e+02,-23.839185,1.147233e+01,-1.236685e+01,-11.115525,-16.944333,8.087975,6.862538,0.593729,-8.488259,-1.112122,-11.806600,0.0,-386.320340,-255.004961,52.752775,-184.068155
4,0.004,1.638818e+02,-23.809284,1.074699e+01,-1.306229e+01,-10.986004,-16.860153,6.112583,8.519467,0.713262,-8.564924,-1.010819,-11.790201,0.0,-286.158169,-272.821316,52.324287,-65.661140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.094,2.386575e+04,768.930000,1.565062e+03,2.333992e+03,-5.096257,-17.700482,781.977516,18.469386,-4.950595,-0.636032,-1.946932,-10.696946,0.0,733.062918,755.965586,67.172172,-90.074840
95,0.095,2.913745e+04,1096.264921,1.910768e+03,3.007033e+03,-6.868815,-18.177258,1037.285745,97.560477,-10.878267,1.056739,-0.892606,-13.546336,0.0,690.993874,696.607832,67.178743,-72.792701
96,0.096,1.328639e+04,4920.934754,8.712917e+02,5.792226e+03,814.146460,-42.638112,3995.416485,93.710263,12.756095,0.479217,0.170192,-13.355594,0.0,-26590.785376,-28856.304504,2144.492333,121.026794
97,0.097,2.059939e+05,346.583543,1.350861e+04,1.385520e+04,122.874236,-26.631931,79.719083,182.810622,-6.709788,-12.390665,1.325561,-9.990932,0.0,-398.072117,-804.604009,529.122065,-122.590173


In [4]:
aa_pdb_path = os.path.join("Input","HIN","AA.pdb")
pdb_aa = openmm.app.PDBFile(aa_pdb_path)
positions_orig = pdb_aa.positions
a = openmm.Vec3(32.0,  0.0,  0.0) * 0.1
b = openmm.Vec3( 0.0, 32.0,  0.0) * 0.1
c = openmm.Vec3( 0.0,  0.0, 32.0) * 0.1


In [5]:
top_aa = openmm.app.GromacsTopFile('AA_gmx.top', periodicBoxVectors=(a,b,c))
system = top_aa.createSystem(nonbondedMethod=openmm.app.PME, nonbondedCutoff=1*openmm.unit.nanometer)
system.removeForce(0)
system.removeForce(0)
system.removeForce(0)
system.removeForce(0)
system.removeForce(0)

frc_a = openmm.ArrowForce()
frc_a.setArbalestConfig("AA_arb_ene.xml")
system.addForce(frc_a)

system.getForces()

[<openmm.openmm.ArrowForce; proxy of <Swig Object of type 'OpenMM::ArrowForce *' at 0x7f9c1fb9e430> >]

In [6]:
integrator = openmm.VerletIntegrator(1.0)
platform = openmm.Platform.getPlatformByName("Reference")
context = openmm.Context(system, integrator, platform)

ReferenceCalcArrowForceKernel::initialize() pt 1 
Initialize Arbalest structures with config file: AA_arb_ene.xml
FFNNAdjustmentTargetGroup::loadNN()  pt 1 
Simulation configuration successfully loaded!
ReferenceCalcArrowForceKernel::initialize() before SimController.m_pSimEnv->OnTaskStarted() pt 10 


2025-01-23 10:02:35.938841: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
context.setPositions(positions_orig)
context.setPeriodicBoxVectors(a, b, c)

state = context.getState(getEnergy=True,getForces=True)
ene=state.getPotentialEnergy()
frc_analyt=state.getForces(asNumpy=True).value_in_unit(openmm.unit.kilocalories_per_mole / openmm.unit.angstrom)
frc_analyt_kJnm=state.getForces(asNumpy=True).value_in_unit(openmm.unit.kilojoules_per_mole / openmm.unit.nanometer)
ene.value_in_unit(openmm.unit.kilocalories_per_mole)

-27.739070550471958

In [8]:
for pos in positions_orig:
    crd = pos.value_in_unit(openmm.unit.angstrom)
    print(f" {crd[0]:9.6f} {crd[1]:9.6f} {crd[2]:9.6f}")

  2.000000  1.000000 -0.000000
  2.000000  2.090000  0.000000
  1.486000  2.454000  0.890000
  1.486000  2.454000 -0.890000
  3.427000  2.641000 -0.000000
  4.391000  1.877000 -0.000000
  3.555000  3.970000 -0.000000
  2.733000  4.556000 -0.000000
  4.853000  4.614000 -0.000000
  5.408000  4.316000  0.890000
  5.661000  4.221000 -1.232000
  5.123000  4.521000 -2.131000
  6.630000  4.719000 -1.206000
  5.809000  3.141000 -1.241000
  4.713000  6.129000  0.000000
  3.601000  6.653000  0.000000
  5.846000  6.835000  0.000000
  6.737000  6.359000 -0.000000
  5.846000  8.284000  0.000000
  4.819000  8.648000  0.000000
  6.360000  8.648000  0.890000
  6.360000  8.648000 -0.890000


In [9]:
frc_analyt

array([[ 2.71625552e+00,  4.21115914e+00, -1.40480479e-02],
       [ 5.13031001e+00,  6.51227087e+00,  2.30431305e-01],
       [ 8.77853248e-01, -3.56872953e-01, -3.10172022e+00],
       [ 8.91918735e-01, -3.58826998e-01,  3.08282307e+00],
       [ 1.16240173e+01, -3.71982149e+01,  7.76996376e-01],
       [-1.15835181e+01, -4.33203318e+00,  1.64012638e+00],
       [ 7.55664245e+00,  4.49376775e+01, -2.69990624e+00],
       [-1.91526774e+00,  2.37473526e+00, -8.49433528e-01],
       [ 8.33590106e+00, -9.15560036e+00, -7.64181184e+00],
       [-1.02455867e+01,  4.19869635e+00,  1.71861024e+00],
       [-3.80797995e+01,  1.60151729e+01,  3.83108932e+01],
       [ 8.64096071e+00, -3.02142748e+00, -2.91232499e+00],
       [ 9.16953605e+00, -6.85253460e+00, -1.63262860e+01],
       [ 1.03849721e+01, -2.45935184e+00, -1.41418431e+01],
       [-3.16807680e+01, -5.06691939e+00,  2.65785272e+00],
       [-9.95658819e-02, -1.90331767e+01, -1.86321350e+00],
       [ 3.12534631e+01,  1.35813545e+01

In [10]:
import copy
delt = 0.0005
na = system.getNumParticles()

frc_num = np.zeros((na,3))

for i in range(na):
    for j in range(3):
        positions = copy.deepcopy(positions_orig)
        increment = np.zeros(3)
        increment[j] = delt
        positions[i] += increment * openmm.unit.nanometer
        
        print(positions[0][0],positions[0][1],positions[0][2])
        
        context.setPositions(positions)
        state = context.getState(getEnergy=True,getForces=True)
        ene_2 = state.getPotentialEnergy().value_in_unit(openmm.unit.kilojoules_per_mole)
        
        positions[i] += (-2.0) * increment * openmm.unit.nanometer        
        context.setPositions(positions)
        state = context.getState(getEnergy=True,getForces=True)
        ene_1 = state.getPotentialEnergy().value_in_unit(openmm.unit.kilojoules_per_mole)
        
        frc_num[i][j] = -(ene_2 - ene_1)/(2.0*delt)
        
frc_num
        

0.2005 nm 0.1 nm 0.0 nm
0.2 nm 0.1005 nm 0.0 nm
0.2 nm 0.1 nm 0.0005 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 nm 0.1 nm -0.0 nm
0.2 

array([[ 1.14009597e+02,  4.45343499e+02, -6.91314730e-01],
       [ 2.13463451e+02, -2.19323007e+02,  9.73652846e+00],
       [ 3.68212423e+01, -1.45326783e+01, -1.33110500e+02],
       [ 3.75210470e+01, -1.46271495e+01,  1.32286902e+02],
       [ 4.87319761e+02, -1.55625073e+03,  3.24210255e+01],
       [-4.85069925e+02, -1.82169449e+02,  6.89287600e+01],
       [ 3.11866246e+02,  1.88354265e+03, -1.16948739e+02],
       [-7.98811143e+01,  9.95724132e+01, -3.54740443e+01],
       [ 3.50977548e+02, -3.83694006e+02, -2.91548693e+02],
       [-4.28667497e+02,  1.75937004e+02,  6.14744957e+01],
       [-1.54868648e+03,  6.68822480e+02,  1.61869017e+03],
       [ 3.61535412e+02, -1.26934487e+02, -1.17344560e+02],
       [ 3.86695385e+02, -2.86921074e+02, -6.87490076e+02],
       [ 4.35492642e+02, -1.05498162e+02, -5.97009097e+02],
       [-1.32602037e+03, -2.11448348e+02,  1.08573367e+02],
       [-4.46735457e+00, -7.95666560e+02, -7.84466599e+01],
       [ 1.30764221e+03,  5.68405931e+02

In [12]:
rmsd = np.sqrt(np.mean(np.square(frc_num - frc_analyt_kJnm)))
print(rmsd)

101.36354615501219
